In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge import Rouge
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91707\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91707\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:


# Read the dataset from CSV
df1 = pd.read_csv('data\data3.csv')
df2 = pd.read_csv('data\data7.csv')

# Concatenate the dataframes vertically
data = pd.concat([df1, df2], axis=0)

data.dropna(subset=['title', 'abstract'], inplace=True)
# Reset the index of the combined dataframe
data.reset_index(drop=True, inplace=True)
#data = data.head(20000)
# Preprocessing

# Preprocessing
def preprocess_text(text):
    if isinstance(text, str):  # Check if the text is a valid string
        return nltk.sent_tokenize(text)
    return []

data['preprocessed_abstract'] = data['abstract'].apply(preprocess_text)

# Filter out empty or missing abstracts
data = data[data['preprocessed_abstract'].apply(len) > 0]

# Train TF-IDF vectorizer
vectorizer = TfidfVectorizer()
sentence_vectors = vectorizer.fit_transform(data['preprocessed_abstract'].apply(' '.join))

In [11]:
for i in range(10):  # Modify the range as needed
    # Choose the abstract and title
    abstract = data.loc[i, 'abstract']
    title = data.loc[i, 'title']

    # Transform the abstract using TF-IDF vectorizer
    abstract_vector = vectorizer.transform([' '.join(preprocess_text(abstract))])

    # Compute cosine similarity
    similarity_scores = cosine_similarity(abstract_vector, sentence_vectors)

    # Get the index of the most similar sentence
    most_similar_index = similarity_scores.argmax()

    # Generate summary
    summary = data['preprocessed_abstract'].iloc[most_similar_index][0]

    # Calculate ROUGE score
    rouge = Rouge()
    rouge_scores = rouge.get_scores(summary, title)

    # Extract the ROUGE-1 F1 score for evaluation
    rouge_1_f1_score = rouge_scores[0]['rouge-1']['f']

    # Print results
    print("Generated Summary:", summary)
    print("Target Summary:", title)
    print("ROUGE-1 F1 Score:", rouge_1_f1_score)
    print("-----------------------------------------")


Generated Summary: In our study, two dairy compost heaps and one vegetable scrap compost heap were constructed on two research farms in Clemson, SC.
Target Summary: A Microbiological Survey and Characterization
ROUGE-1 F1 Score: 0.07999999680000014
-----------------------------------------
Generated Summary: We describe geometric non-commutative formal groups in terms of a geometric commutative formal group with a Poisson structure on its splay algebra.
Target Summary: eb 2 00 6 Non-Commutative Formal Groups in Positive Characteristic
ROUGE-1 F1 Score: 0.06896551272294917
-----------------------------------------
Generated Summary: In the numerical solution of partial differential equations (PDEs), high-quality meshes are crucial for the stability, accuracy, and convergence of the associated PDE solver.
Target Summary: An Alternating Mesh Quality Metric Scheme for Efficient Mesh Quality Improvement
ROUGE-1 F1 Score: 0.06666666246666693
-----------------------------------------
Generate

KeyboardInterrupt: 